In [1]:
# import libraries
import pandas as pd
import sqlite3


import numpy as np
from numpy import std, mean, sqrt
import scipy.stats as stats
from statsmodels.stats.power import tt_ind_solve_power
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [2]:
# connect to database
conn = sqlite3.Connection('Northwind_large.sqlite')
cur = conn.cursor()

In [3]:
#what tables are available in the database
cur.execute("""SELECT name 
               FROM sqlite_master
               WHERE type = 'table';""")
cur.fetchall()

[('Employee',),
 ('Category',),
 ('Customer',),
 ('Shipper',),
 ('Supplier',),
 ('Order',),
 ('Product',),
 ('OrderDetail',),
 ('CustomerCustomerDemo',),
 ('CustomerDemographic',),
 ('Region',),
 ('Territory',),
 ('EmployeeTerritory',),
 ('sqlite_sequence',)]

In [4]:
# OrderDetail data

cur.execute("""SELECT Id AS CustomerID, ContactName AS CustomerName, City, Country, Region
               FROM Customer
               """)
df_Customer= pd.DataFrame(cur.fetchall())
df_Customer.columns = [x[0] for x in cur.description]
df_Customer.sample(10)

,CustomerID,CustomerName,City,Country,Region
52,NORTS,Simon Crowther,London,UK,British Isles
11,CACTU,Patricio Simpson,Buenos Aires,Argentina,South America
34,HILAA,Carlos Hernández,San Cristóbal,Venezuela,South America
46,LINOD,Felipe Izquierdo,I. de Margarita,Venezuela,South America
73,SPECD,Dominique Perrier,Paris,France,Western Europe
2,ANTON,Antonio Moreno,México D.F.,Mexico,Central America
65,REGGC,Maurizio Moroni,Reggio Emilia,Italy,Southern Europe
69,SANTG,Jonas Bergulfsen,Stavern,Norway,Scandinavia
24,FRANK,Peter Franken,München,Germany,Western Europe
55,OTTIK,Henriette Pfalzheim,Köln,Germany,Western Europe


### Manage n/a data types<a id='Manage n/a data types'></a>

In [46]:
print(df_Customer)

   CustomerID                     Name         City  Country           Region
0       ALFKI             Maria Anders       Berlin  Germany   Western Europe
1       ANATR             Ana Trujillo  México D.F.   Mexico  Central America
2       ANTON           Antonio Moreno  México D.F.   Mexico  Central America
3       AROUT             Thomas Hardy       London       UK    British Isles
4       BERGS       Christina Berglund        Luleå   Sweden  Northern Europe
..        ...                      ...          ...      ...              ...
86      WARTH         Pirkko Koskitalo         Oulu  Finland      Scandinavia
87      WELLI            Paula Parente      Resende   Brazil    South America
88      WHITC           Karl Jablonski      Seattle      USA    North America
89      WILMK          Matti Karttunen     Helsinki  Finland      Scandinavia
90      WOLZA  Zbyszek Piestrzeniewicz     Warszawa   Poland   Eastern Europe

[91 rows x 5 columns]


### Remove duplicate data<a id='Remove duplicate data'></a>

In [48]:
# identification of duplicate data
print('duplicated rows: ' + str(df_Customer.duplicated().sum()))

duplicated rows: 0


In [50]:
df_Customer.columns = df_Customer.columns.str.strip()

### Manage data types<a id='Manage data types'></a>

In [51]:
# data type identification
df_Customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  91 non-null     object
 1   Name        91 non-null     object
 2   City        91 non-null     object
 3   Country     91 non-null     object
 4   Region      91 non-null     object
dtypes: object(5)
memory usage: 3.7+ KB


### Remove outliers<a id='Remove outliers'></a>

In [52]:
df_Customer.describe()

,CustomerID,Name,City,Country,Region
count,91,91,91,91,91
unique,91,91,69,21,9
top,ALFKI,Maria Anders,London,USA,Western Europe
freq,1,1,6,13,28


In [53]:
df_Customer.to_csv(r'Customer.csv')